# Import libraries

In [ ]:
import os
import yaml
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib
matplotlib.rcParams.update({'font.size': 8})

from obspy.core import Stream, Trace, UTCDateTime, Stats
from obspy.io.sac import SACTrace

# Event

In [ ]:
# read event location
with open('./simu_with_3d_crust/input/inparam.source.yaml', 'r') as file:
    source_yaml = yaml.load(file, Loader=yaml.FullLoader)
loc_leaf = source_yaml['list_of_sources'][0]['VIRGINIA_201108231751A']['location']
event_latlon = loc_leaf['latitude_longitude']
event_depth = loc_leaf['depth']

# USArray

### Station info

In [ ]:
# read station locations
info_US_TA = np.loadtxt('./simu_with_3d_crust/input/US_TA.txt', dtype=str, skiprows=3)

# dict: station key -> [lat, lon]
nstation = len(info_US_TA)
stlatlon_dict = {}
for ist in np.arange(nstation):
    key = info_US_TA[ist, 1] + '.' + info_US_TA[ist, 0]
    stlatlon_dict[key] = np.array([float(info_US_TA[ist, 2]), float(info_US_TA[ist, 3])])

### Seismograms

In [ ]:
def read_us_ta(us_ta_dir, channel = 'U3'):
    # read rank-station info
    rank_station_info = np.loadtxt(us_ta_dir + '/rank_station.info', dtype=str, skiprows=1)

    # dict: mpi-rank -> [station keys]
    rank_station_dict = {}

    # (name, lat, lon) of stations re-ordered by data
    stlatlon_in_data_order = []
    stname_in_data_order = []

    for item in rank_station_info:
        rank = item[0]
        stkey = item[1]
        # initialize with an empty array if rank does not exists in rank_station_dict
        if rank not in rank_station_dict.keys():
            rank_station_dict[rank] = []
        # append the station
        rank_station_dict[rank].append(stkey)
        stlatlon_in_data_order.append(stlatlon_dict[stkey])
        stname_in_data_order.append(stkey)
    stlatlon_in_data_order = np.array(stlatlon_in_data_order)

    # read time
    time = np.loadtxt(us_ta_dir + '/data_time.ascii')
    ntime = len(time)

    # allocate data
    data = np.ndarray((ntime, nstation))

    # loop over mpi-ranks to read data
    pos = 0
    for rank in rank_station_dict.keys():
        data_on_rank = np.loadtxt('%s/dir_rank%s/%s.ascii' % (us_ta_dir, rank, channel))
        data[:, pos:pos + len(rank_station_dict[rank])] = data_on_rank
        pos += len(rank_station_dict[rank])
    
    return time, data, stlatlon_in_data_order, stname_in_data_order


time_1d, data_1d, stlatlon_in_data_order_1d, stname_in_data_order_1d = \
read_us_ta('./simu_with_1d_crust/output/stations/USArray_transportable')
time_3d, data_3d, stlatlon_in_data_order_3d, stname_in_data_order_3d = \
read_us_ta('./simu_with_3d_crust/output/stations/USArray_transportable')

In [ ]:
n_seis = 100
np.random.seed(0)
indexes = np.random.choice(data_1d.shape[1], n_seis)

# plot seismograms
plt.figure(dpi=200, figsize=(10, n_seis / 2))
for i, ind in enumerate(indexes):
    end = None
    norm = np.max(np.abs(data_1d[:end, ind]))
    plt.plot(time_1d[:end], data_1d[:end, ind] / norm + i * 2, c='k', label='1D crust')
    plt.plot(time_3d[:end], data_3d[:end, ind] / norm + i * 2, c='r', label='3D crust')
    plt.text(-200, i * 2, stname_in_data_order_1d[ind], ha='left', va='bottom')
plt.ylim(-2, n_seis * 2)
plt.yticks([])
plt.show()

### Animations on array

In [ ]:
#############################
###### plot a snapshot ######
#############################

# specify a time step (0~384)
tstep = 100
channel = 'U3'

# plot the snapshot
plt.figure(dpi=150)
plt.gca().axis('off')
plt.scatter(stlatlon_in_data_order_3d[:, 1], stlatlon_in_data_order_3d[:, 0], s=1, 
            c=data_3d[tstep, :], vmin=-1e-6, vmax=1e-6, cmap='coolwarm')
plt.text(0, 0, 'Time = %.1f s' % (time_3d[tstep]), transform = plt.gca().transAxes)
plt.colorbar(orientation='vertical', shrink=.5, label=channel)
plt.gca().set_aspect(1.3)
plt.show()

In [ ]:
############################
###### make animation ######
############################

def animate(us_ta_dir, time, data, stlatlon_in_data_order, norm=1e-6):
    # create dir
    os.makedirs(us_ta_dir + '/animation', exist_ok=True)

    # create all snapshots
    print('Making snapshots...')
    for tstep in np.arange(len(time)):
        print('%d / %d' % (tstep + 1, len(time)), end='\r')
        plt.figure(dpi=150)
        plt.gca().axis('off')
        plt.scatter(stlatlon_in_data_order[:, 1], stlatlon_in_data_order[:, 0], s=1, 
                    c=data[tstep, :], vmin=-norm, vmax=norm, cmap='coolwarm')
        plt.text(0, 1, 'Time = %.1f s' % (time[tstep]), transform = plt.gca().transAxes)
        plt.colorbar(orientation='vertical', shrink=.5, label=channel)
        plt.gca().set_aspect(1.3)
        plt.savefig(us_ta_dir + '/animation/%s.%04d.png' % (channel, tstep))
        plt.close()

    # use ffmepg to combine snapshots to animation
    print('Creating video using ffmpeg...')
    os.system("ffmpeg -y -i %s/animation/%s.%%04d.png %s/animation/%s.mp4" % 
              (us_ta_dir, channel, us_ta_dir, channel))

    # remove snapshots
    os.system('rm ' + us_ta_dir + '/animation/%s.*.png' % (channel,))
    print('Done.')
    
animate('./simu_with_1d_crust/output/stations/USArray_transportable', time_1d, data_1d, stlatlon_in_data_order_1d)
animate('./simu_with_3d_crust/output/stations/USArray_transportable', time_3d, data_3d, stlatlon_in_data_order_3d)

In [ ]:
# play animation
from IPython.display import Video
Video("%s/animation/%s.mp4" % ('./simu_with_1d_crust/output/stations/USArray_transportable', channel))

In [ ]:
Video("%s/animation/%s.mp4" % ('./simu_with_3d_crust/output/stations/USArray_transportable', channel))